<a href="https://colab.research.google.com/github/Batwan01/2024-Challenge/blob/main/history/24-8-31/MLP_Kfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import json
import numpy as np
import random
import pandas as pd

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import ParameterGrid
from google.colab import drive

drive.mount('/content/drive')

# 재현 가능성을 위한 시드 고정
RANDOM_SEED = 18
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Mounted at /content/drive


In [2]:
# 데이터 경로 설정
train_csv_path = "/content/drive/MyDrive/Colab Notebooks/contest/samsung/train.csv"
test_csv_path = "/content/drive/MyDrive/Colab Notebooks/contest/samsung/test.csv"
submission_csv_path = '/content/drive/MyDrive/Colab Notebooks/contest/samsung/sample_submission.csv'

In [21]:
# MLP - 은닉층 5개
# 현재 최고성능 모델 하이퍼파라미터(GS) : {"batch_size": 32, "hidden_sizes": [32, 64, 64, 32, 16], "learning_rate": 0.001}
# Epoch : 93
# 제출 성능 : 0.752
class MLP5Hidden(nn.Module):
    def __init__(self, input_size=22, hidden_sizes=[32, 64, 64, 32, 16], output_size=1):
        super(MLP5Hidden, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.fc4 = nn.Linear(hidden_sizes[2], hidden_sizes[3])
        self.fc5 = nn.Linear(hidden_sizes[3], hidden_sizes[4])
        self.fc6 = nn.Linear(hidden_sizes[4], output_size)
        self.lrelu = nn.LeakyReLU()

    def forward(self, x):
        x = self.lrelu(self.fc1(x))
        x = self.lrelu(self.fc2(x))
        x = self.lrelu(self.fc3(x))
        x = self.lrelu(self.fc4(x))
        x = self.lrelu(self.fc5(x))
        x = self.fc6(x)
        return x

In [22]:
# 잔차 연결
class ResidualMLP(nn.Module):
    def __init__(self, input_size=22, hidden_sizes=[32, 64, 64, 32, 16], output_size=1):
        super(ResidualMLP, self).__init__()

        # 6개의 MLP 레이어 정의
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.prelu1 = nn.PReLU()

        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.prelu2 = nn.PReLU()

        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.prelu3 = nn.PReLU()

        self.fc4 = nn.Linear(hidden_sizes[2], hidden_sizes[3])
        self.prelu4 = nn.PReLU()

        self.fc5 = nn.Linear(hidden_sizes[3], hidden_sizes[4])
        self.prelu5 = nn.PReLU()

        self.fc6 = nn.Linear(hidden_sizes[4], output_size)
        self.prelu6 = nn.PReLU()

        # 1x1 Linear 레이어로 Residual Connection의 크기 맞추기
        self.residual1 = nn.Linear(hidden_sizes[0], hidden_sizes[2])  # fc1 -> fc3
        self.residual2 = nn.Linear(hidden_sizes[2], hidden_sizes[4])  # fc3 -> fc5

    def forward(self, x):
        # 첫 번째 MLP 레이어
        out = self.prelu1(self.fc1(x))

        # 첫 번째 Residual 연결 (fc1 -> fc2 -> fc3)
        residual = self.residual1(out)  # 크기 맞추기
        out = self.prelu2(self.fc2(out))
        out = self.prelu3(self.fc3(out))
        out += residual  # 첫 번째 Residual Connection

        # 두 번째 Residual 연결 (fc3 -> fc4 -> fc5)
        residual = self.residual2(out)  # 크기 맞추기
        out = self.prelu4(self.fc4(out))
        out = self.prelu5(self.fc5(out))
        out += residual  # 두 번째 Residual Connection

        # 최종 출력 레이어 (fc6) 및 PReLU 적용
        out = self.prelu6(self.fc6(out))  # 최종 레이어에도 PReLU 적용

        return out

In [23]:
# 훈련 데이터를 훈련 + 검증 데이터로 나누고 train_dataset, val_dataset 을 반환하는 함수:
train_data = pd.read_csv(train_csv_path)

X = torch.tensor(train_data.iloc[:,1:-1].values, dtype=torch.float32)
y = torch.tensor(train_data.iloc[:,-1].values, dtype = torch.float32).view(-1,1)
print(X.shape)

# Square term 추가하기 (원본 X에 각 특징의 제곱값 추가)
X_square = X ** 2
X = torch.cat((X, X_square), dim=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

print(X_train.shape)

torch.Size([40118, 11])
torch.Size([32094, 22])


In [33]:
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, TensorDataset

# KFold 객체 생성
kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

# K-Fold 교차 검증 수행
fold = 1
for train_index, val_index in kf.split(X):
    print(f'Fold {fold}')

    # 훈련/검증 데이터 분할
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, worker_init_fn=worker_init_fn)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, worker_init_fn=worker_init_fn)

    # 모델, 손실 함수, 옵티마이저 초기화
    model = ResidualMLP(input_size=X_train.shape[1])
    model.cuda()
    criterion = nn.MSELoss().cuda()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # 훈련 루프
    train_losses = []
    val_losses = []
    for epoch in range(60):
        model.train()
        train_loss = 0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.cuda(), batch_y.cuda()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss = train_loss / len(train_loader)
        train_losses.append(train_loss)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.cuda(), batch_y.cuda()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()

        val_loss = val_loss / len(val_loader)
        val_losses.append(val_loss)

        print(f'Epoch {epoch+1}/60, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

    fold += 1

Fold 1
Epoch 1/60, Train Loss: 225.3904, Validation Loss: 3.8880
Epoch 2/60, Train Loss: 3.4511, Validation Loss: 3.7620
Epoch 3/60, Train Loss: 3.4384, Validation Loss: 3.6996
Epoch 4/60, Train Loss: 3.4008, Validation Loss: 3.8048
Epoch 5/60, Train Loss: 3.3985, Validation Loss: 3.5892
Epoch 6/60, Train Loss: 3.4335, Validation Loss: 3.7513
Epoch 7/60, Train Loss: 3.4466, Validation Loss: 5.8041
Epoch 8/60, Train Loss: 3.2484, Validation Loss: 3.4421
Epoch 9/60, Train Loss: 3.2427, Validation Loss: 3.5823
Epoch 10/60, Train Loss: 3.3528, Validation Loss: 3.9151
Epoch 11/60, Train Loss: 3.1157, Validation Loss: 3.3261
Epoch 12/60, Train Loss: 3.2093, Validation Loss: 3.3187
Epoch 13/60, Train Loss: 3.2749, Validation Loss: 3.6540
Epoch 14/60, Train Loss: 3.2192, Validation Loss: 3.4329
Epoch 15/60, Train Loss: 3.1984, Validation Loss: 4.4100
Epoch 16/60, Train Loss: 3.1124, Validation Loss: 3.4433
Epoch 17/60, Train Loss: 3.1637, Validation Loss: 3.3485
Epoch 18/60, Train Loss: 3.0983

In [34]:
X_test = pd.read_csv(test_csv_path).iloc[:,1:]

# Square term 추가하기
X_test_square = X_test ** 2
X_test = pd.concat([X_test, X_test_square], axis=1)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

test_dataset = TensorDataset(X_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()
test_pred = []
with torch.no_grad():
    for batch in test_loader:
        batch_X = batch[0].cuda()
        outputs = model(batch_X)
        test_pred.extend(outputs.cpu().numpy())  # 예측값을 CPU로 이동하여 리스트에 추가

test_pred = np.array(test_pred).flatten()

submission_df = pd.read_csv(submission_csv_path)
submission_df['y'] = test_pred  # 예측 결과를 y 컬럼에 추가

# 지정된 경로에 CSV 파일로 저장
submission_df.to_csv("/content/drive/MyDrive/Colab Notebooks/contest/samsung/result/MLP_square_term_Kfold.csv", index=False)

In [36]:
submission_df.describe()

,y
count,4986.000000
mean,85.929970
std,3.494783
min,82.263710
25%,83.408710
50%,84.286884
75%,87.501263
max,95.067207


In [35]:
import pandas as pd

def find_non_matching_ids(file1, file2):
    # 두 개의 CSV 파일을 읽어옴
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    # y 값 기준으로 내림차순 정렬
    df1_sorted = df1.sort_values(by='y', ascending=False)
    df2_sorted = df2.sort_values(by='y', ascending=False)

    # file1의 상위 10% 항목 계산
    top_10_percent_count = int(len(df1_sorted) * 0.1)
    top_10_percent_ids_df1 = set(df1_sorted.head(top_10_percent_count)['ID'])

    # file2의 상위 10% ID 추출
    top_10_percent_ids_df2 = set(df2_sorted.head(top_10_percent_count)['ID'])

    # file1의 상위 10% 중 file2의 상위 10%에 없는 ID 계산
    non_matching_ids = top_10_percent_ids_df1 - top_10_percent_ids_df2
    num_non_matching = len(non_matching_ids)

    # 결과 출력
    print(f"file1의 상위 10% 항목 개수: {top_10_percent_count}")
    print(f"file1의 상위 10% 중 file2에 없는 항목 개수: {num_non_matching}")
    print(f"file1의 상위 10% 중 file2에 없는 항목 ID: {non_matching_ids}")

    return top_10_percent_count, num_non_matching, list(non_matching_ids)

# 사용 예시
file1 ='/content/drive/MyDrive/Colab Notebooks/contest/samsung/MLP_Residual_Connection_18.csv' # best 성능 파일
file2 = '/content/drive/MyDrive/Colab Notebooks/contest/samsung/result/MLP_square_term_Kfold.csv' # 측정하고자 하는 파일
top_10_percent_count, num_non_matching, non_matching_ids = find_non_matching_ids(file1, file2)


file1의 상위 10% 항목 개수: 498
file1의 상위 10% 중 file2에 없는 항목 개수: 11
file1의 상위 10% 중 file2에 없는 항목 ID: {'TEST_3811', 'TEST_1321', 'TEST_1178', 'TEST_3172', 'TEST_0784', 'TEST_0037', 'TEST_1309', 'TEST_1478', 'TEST_1502', 'TEST_2275', 'TEST_4318'}
